In [40]:
# Working directory
import os
os.chdir('/Users/Josealanis/Documents/Experiments/dpx_tt/eeg/')

#import numpy as np
import mne
from mne.preprocessing import ICA
from mne.preprocessing import create_eog_epochs

In [41]:
#%% READ IN THE DATA

# EEG montage
montage = mne.channels.read_montage(kind = 'biosemi64')

#cd '~/Documents/Experiments/dpx_tt/eeg/bdfs/'
data_path = './bdfs/data2.bdf'

# Import raw data
raw = mne.io.read_raw_edf(data_path, 
                          montage = montage, 
                          preload = True, 
                          stim_channel = -1, 
                          exclude = ['EOGH_rechts', 'EOGH_links', 
                                     'EXG5', 'EXG6', 'EXG7', 'EXG8'])

# Get data information
raw.info


Extracting EDF parameters from ./bdfs/data2.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
The following EEG sensors did not have a position specified in the selected montage: ['EOGV_oben', 'EOGV_unten']. Their position has been left untouched.
Reading 0 ... 1034495  =      0.000 ...  4040.996 secs...


<ipython-input-41-ff2ba4946600>:15: RuntimeWarning: The following EEG sensors did not have a position specified in the selected montage: ['EOGV_oben', 'EOGV_unten']. Their position has been left untouched.
  'EXG5', 'EXG6', 'EXG7', 'EXG8'])


<Info | 18 non-empty fields
    bads : list | 0 items
    buffer_size_sec : float | 1.0
    ch_names : list | Fp1, AF7, AF3, F1, F3, F5, F7, FT7, FC5, ...
    chs : list | 67 items (EEG: 66, STIM: 1)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : Transform | 3 items
    dig : list | 67 items
    events : list | 0 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 52.0 Hz
    meas_date : int | 1434474755
    nchan : int | 67
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 256.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    description : NoneType
    dev_ctf_t : NoneType
    experimenter : NoneType
    file_id : NoneType
    gantry_angle : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    meas_id : NoneType
    proj_id : NoneType
    proj_name : NoneType
    subject_info :

In [3]:
#%% EDIT INFORMATION

# Note the sampling rate of recording
sfreq = raw.info['sfreq']
sfreq = int(sfreq)
# and Buffer size ???
bsize = raw.info['buffer_size_sec']

# Channel names
chans = raw.info['ch_names'][0:64]
chans.extend(['EXG1', 'EXG2', 'Stim'])

# Write a list of channel types (e.g., eeg, eog, ecg)
chan_types = ['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 
              'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 
              'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 
              'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 
              'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 
              'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 
              'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 
              'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 
              'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 
              'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 
              'eeg', 'eeg', 'eeg', 'eeg', 'eeg',
              'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 
              'eeg', 'eeg', 'eeg', 'eeg',
              'eog', 'eog', 'stim']

# Bring it all together with 
# MNE.function for creating custom EEG info files
info_custom = mne.create_info(chans, sfreq, chan_types, montage)

# You also my add a short description of the data set
info_custom['description'] = 'DPX Baseline'

# Replace the mne info structure with the customized one 
# which has the correct labels, channel types and positions.
raw.info = info_custom
raw.info['buffer_size_sec'] = bsize

# check data information 
raw.info

<Info | 18 non-empty fields
    bads : list | 0 items
    buffer_size_sec : float | 1.0
    ch_names : list | Fp1, AF7, AF3, F1, F3, F5, F7, FT7, FC5, ...
    chs : list | 67 items (EEG: 64, EOG: 2, STIM: 1)
    comps : list | 0 items
    custom_ref_applied : bool | False
    description : str | 12 items
    dev_head_t : Transform | 3 items
    dig : list | 67 items
    events : list | 0 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 128.0 Hz
    nchan : int | 67
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 256.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    dev_ctf_t : NoneType
    experimenter : NoneType
    file_id : NoneType
    gantry_angle : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    meas_date : NoneType
    meas_id : NoneType
    proj_id : NoneType
    proj_name : NoneType
    subject

In [4]:
#%% GET EVENT INFORMATION
# Next, define the type of data you have provided in 'picks'
picks = mne.pick_types(raw.info, 
                       meg = False, 
                       eeg = True, 
                       eog = True,
                       stim = True)


# EVENTS
events = mne.find_events(raw, 
                         stim_channel = 'Stim', 
                         output = 'onset', 
                         min_duration = 0.002)

# EVENTS THAT ARE CUE STIMULI
evs = events[(events[:,2] >= 70) & (events[:,2] <= 75), ]
# LATENCIES
latencies = events[(events[:,2] >= 70) & (events[:,2] <= 75), 0]
# DIFFERENCE BETWEEN TWO CONSEQUITIVE CUES
diffs = [x-y for x, y in zip(latencies, latencies[1:])]
# GET FIRST EVENT AFTER BREAKS (pauses between blocks,
# time difference between two events is > 10 seconds)
diffs  = [abs(number)/sfreq for number in diffs]
breaks = [i+1 for i in range(len(diffs)) if diffs[i] > 10]

# start first block
b1s = ( latencies[ breaks[0] ] - (2*sfreq) ) / sfreq
# end of frist block
b1e = ( latencies[ (breaks[1]-1) ] + (6*sfreq) ) / sfreq

# start second block
b2s = ( latencies[ breaks[1] ] - (2*sfreq) ) / sfreq
# end of frist block
b2e = ( latencies[ (breaks[2]-1) ] + (6*sfreq) ) / sfreq

Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
1757 events found
Event IDs: [   12    13    70    71    72    73    74    75    76    77    78    79
    80    81   112   113   127   245 65536 65649 65662]


In [28]:
# Block infos
print('Block 1 from', round(b1s, 3), 'to', round(b1e, 3), '    Block lenght ', round(b1e-b1s, 3))
print('Block 2 from', round(b2s, 3), 'to', round(b2e, 3), '    Block lenght ', round(b2e-b2s, 3))


Block 1 from 758.551 to 1479.496     Block lenght  720.945
Block 2 from 1553.004 to 2273.297     Block lenght  720.293


In [36]:
#%% Concatenate blocks data
# Block 1
raw_bl1 = raw.copy().crop(tmin = b1s, tmax = b1e)
# Block 2
raw_bl2 = raw.copy().crop(tmin = b2s, tmax = b2e)

# Bind them together
raw_blocks = mne.concatenate_raws([raw_bl1, raw_bl2])

In [37]:
raw_bl1

<RawEDF  |  data2.bdf, n_channels x n_times : 67 x 368959 (1441.2 sec), ~188.8 MB, data loaded>

In [33]:
raw_bl2

<RawEDF  |  data2.bdf, n_channels x n_times : 67 x 184396 (720.3 sec), ~94.4 MB, data loaded>

In [ ]:
#%% CHECK if data ok
%matplotlib notebook
raw_blocks.plot(scalings = dict(eeg=50e-6), n_channels = 66)

In [38]:
#%% NUMBER OF TRIALS IN CONCATENATED DATASET
keeps = mne.find_events(raw_blocks, 
                        stim_channel = 'Stim', 
                        output = 'onset', 
                        min_duration=0.002)

len(keeps[(keeps[:,2] >= 70) & (keeps[:,2] <= 75), ])

774 events found
Event IDs: [ 12  13  70  71  72  73  74  75  76  77  78  79  80  81 112 113]


258

In [ ]:
#%% FILTER and REREFERENCE the data to reduce noise and remove artefact frequencies.

# Apply filter and average reference
raw_blocks.filter(0.1, 50., 
                  n_jobs = 1, 
                  fir_design = 'firwin') 
raw_blocks.set_eeg_reference(ref_channels = 'average', 
                             projection = False)

In [ ]:
#%% ICA DECOMPOSITION

# Define electrodes to include in ICA
picks = mne.pick_types(raw_blocks.info, 
                       meg = False, 
                       eeg = True, 
                       eog = False,
                       stim = False)
n_components = 25
method = 'extended-infomax'
#decim = 3
reject = None
ica = ICA(n_components=n_components, method=method)
ica.fit(raw_blocks.copy().filter(1,50), picks=picks, reject = dict(eeg = 3e-4))
#ica.fit(raw.copy().filter(1,50), picks=picks, reject=reject) 

In [39]:
#raw_blocks.info

<Info | 18 non-empty fields
    bads : list | 0 items
    buffer_size_sec : float | 1.0
    ch_names : list | Fp1, AF7, AF3, F1, F3, F5, F7, FT7, FC5, ...
    chs : list | 67 items (EEG: 64, EOG: 2, STIM: 1)
    comps : list | 0 items
    custom_ref_applied : bool | False
    description : str | 12 items
    dev_head_t : Transform | 3 items
    dig : list | 67 items
    events : list | 0 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 128.0 Hz
    nchan : int | 67
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 256.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    dev_ctf_t : NoneType
    experimenter : NoneType
    file_id : NoneType
    gantry_angle : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    meas_date : NoneType
    meas_id : NoneType
    proj_id : NoneType
    proj_name : NoneType
    subject

In [ ]:
ica.plot_components()